<a href="https://colab.research.google.com/github/ldalomoto/Proyecto_Final_Desarrollo/blob/main/Skills.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
torch.cuda.is_available(), torch.cuda.get_device_name(0)

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
!pip install -U transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 13.9 MB/s eta 0:00:00


In [ ]:
!pip install -U sentencepiece

In [ ]:
import json
import hashlib
from pathlib import Path
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
INPUT_DIR = Path("/content/data_unificada")
OUTPUT_DIR = Path("/content/data_skills_final")
INPUT_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from huggingface_hub import login
login()

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

MODEL_NAME = "meta-llama/Meta-Llama-3-8B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto"
)

print("✅ Modelo cargado correctamente")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
def gen_id(prefix, text):
    base = f"{prefix}_{text}".encode("utf-8")
    return f"{prefix}_{hashlib.sha1(base).hexdigest()[:10]}"

def safe(value, default=""):
    return value if value is not None else default

def limpiar_skill(texto):
    return texto.strip().lower()

In [ ]:
def generar_prompt_materia(career, subject_name):
    return f"""<|begin_of_text|>
<|system|>
Eres un experto en diseño curricular universitario en Ecuador.
Respondes únicamente en JSON válido, sin texto adicional.
<|end|>

<|user|>
Genera habilidades profesionales que desarrolla un estudiante al aprobar
la siguiente materia universitaria.

Materia: {subject_name}
Carrera: {career['career_name']}
Universidad: {career['university_name']}
Descripción de la carrera:
{career.get('description', '')[:500]}

Reglas:
- Idioma: español
- 8 a 12 habilidades
- Redacción profesional
- No usar placeholders
- No repetir texto
- Responde SOLO un objeto JSON

Formato exacto:
{{
  "skills": [
    "habilidad concreta y profesional"
  ]
}}
<|end|>

<|assistant|>
"""

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
def inferir_skills(prompt):
    inputs = tokenizer(
      prompt,
      return_tensors="pt",
      truncation=True,
      max_length=2048,
      padding=True
    )

    inputs = {k: v.to(DEVICE) for k, v in inputs.items()}

    outputs = model.generate(
        **inputs,
        max_new_tokens=250,
        do_sample=False,
        temperature=0.2,
        repetition_penalty=1.15,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id
    )

    texto = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 🔥 EXTRAER SOLO EL PRIMER JSON
    if "<|assistant|>" in texto:
        texto = texto.split("<|assistant|>")[-1].strip()

    start = texto.find("{")
    end = texto.rfind("}") + 1


    if start == -1 or end == -1:
        return []

    json_text = texto[start:end]

    try:
        data = json.loads(json_text)
        return data.get("skills", [])
    except json.JSONDecodeError:
        return []

In [ ]:
def main():
    resultado_final = []

    for archivo in INPUT_DIR.glob("*.json"):
        print(f"\n📂 Procesando archivo: {archivo.name}")

        with open(archivo, "r", encoding="utf-8") as f:
            careers = json.load(f)

        for career in careers:
            career_name = safe(career.get("career_name"))
            career_id = safe(career.get("career_id"), gen_id("career", career_name))
            university = safe(career.get("university_name"))
            locations = career.get("locations", [])

            print(f"🎓 Carrera: {career_name}")

            subjects_out = []
            all_skills = set()

            for subject in career.get("subjects", []):
                subject_name = safe(subject.get("name"))
                if not subject_name:
                    continue

                print(f"  🔹 Materia: {subject_name}")

                prompt = generar_prompt_materia(career, subject_name)
                skills = inferir_skills(prompt)

                print(f"    ⚡ Generando skills")
                print(f"      - {len(skills)} skills encontradas")

                if not skills:
                    continue

                subject_id = gen_id("subject", career_id + subject_name)
                skills_objs = []

                for sk in skills:
                    sk_clean = limpiar_skill(sk)
                    skill_id = gen_id("skill", sk_clean)

                    skills_objs.append({
                        "skill_id": skill_id,
                        "name": sk_clean
                    })
                    all_skills.add(sk_clean)

                subjects_out.append({
                    "subject_id": subject_id,
                    "code": safe(subject.get("code")),
                    "name": subject_name,
                    "semester": safe(subject.get("semester")),
                    "skills": skills_objs
                })

                print(f"    ✅ Skills generadas")

            resultado_final.append({
                "career_id": career_id,
                "career_name": career_name,
                "university_name": university,
                "country": "ECUADOR",
                "locations": locations,
                "subjects": subjects_out,
                "career_general_skills": [
                    {
                        "skill_id": gen_id("skill", sk),
                        "name": sk
                    }
                    for sk in sorted(all_skills)
                ]
            })

    output_file = OUTPUT_DIR / "careers_skills_ecuador.json"
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(resultado_final, f, indent=2, ensure_ascii=False)

    print(f"\n🚀 Proceso terminado → {output_file}")


In [ ]:
if __name__ == "__main__":
    main()